# INF 510_Fall 2019_Final Project 

1.	**The names of team member(s)**:

    Yue Hao.

2.	**How to run your code (what command-line switches they are, what happens when you invoke the code, etc.)**
    
    This project requires the following packages:
    - pandas, numpy, seaborn, folium ,matplotlib 3.1.0 
    
    To make sure that all the graphs will be created successfully, user should check whether these packages have been installed or not. If not, these seaborn and folium packages should be downloaded and installed in advance. There are several ways to do it.In conda prompt, using commend: ***conda create -f environment.yml*** to create a new conda environment. If it fails, using the commend: ***conda install seaborn*** and ***conda install -c conda-forge folium*** to install two packages. Or, in the terminal, using commend: ***pip install seaborn*** and ***pip install folium***. To make sure that the graph will be displayed in a full version, matolptlib version should be 3.1.0.
    To run this project, make sure the above packages are installed, and then simply clone the repo at http://www.github.com/YueHao96/inf510_project and execute this notebook.
    
    If you want to run the python files, there are two offered parameters.
    In terminal, run the commend: ***python main.py relation***, after few minutes, four graphs will be created and saved under the current path. However, these four graphs will not present together. After running the code, one graph show in a pop up window After closing the current window, the next graph will show up. The first graph is a correlation matrix, and the next three graphs are scatter diagrams in four dimensions.
    If commend: ***python main.py distribution*** is used, one map will be presented on the browser and to .html files will be saved under the current path. This map show the distribution of 50 house locations with highest housing values and 50 house locations with lowest housing values in Los Angeles County. On the webpage, this map is interactive. It means that viewer can zoom in or out the map to explore the location of each house.Also, if click on the icon, specific hosing value will pop up.

3.	**Any major “gotchas” to the code (i.e. things that don’t work, go slowly, could be improved, etc.)**
    
   - Firstly, collected data are not adequate enough. Data collection and organization are based on zip codes. Zip codes in Los Angeles County ranges from 90001 to 93599. If available, more than 3000 records for each data set are expected. However, based on zip code and city name, Zillow API is constructed but clean data obtained from requests are only around 250 records. Compared with 3000 expected results, the data volume is not satisfying. It may be explained by the restriction of the website. However, there may be better resources for data collection.
   - Secondly, if city boundaries are drown on the distribution map, it may be easier to view and analysis spatial patterns in city level. Also, two labeled city names are overlapped by the color bar but I do not know how to adjust it.  
   - Thirdly, as I have mentioned in second question, the matplotlib version should be 3.1.0. If the current version of matplotlib is 3.1.1, the correlation matrix will be drawn but the image is not complete. This is a problem that is out of scope to solve.

4.  **Anything else you feel is relevant to the grading of your project your project.**

    When running the jupyter notebook or python files, the database final_project.db should under the same folder with other python files.Also, when running the python files, after closing the current graph, next grap will pop up.
    

5.  **What did you set out to study?**
    
    Major goal of this study is to find out correlations between housing values, surrounding environment and personal income status in Los Angeles County. Also, it aims at finding out specific patterns on spatial distribution of highest and lowest housing values in Los Angeles County.

6. **What did you Discover/what were your conclusions (i.e. what were your findings?  Were your original assumptions confirmed, etc.?)**

    Based on Milestone1, there are three major findings.
    - Firstly, there is a strong positive correlation between household median income and housing value.It is somehow confirmed with the assumption in milestone1 that in areas where housing values are high, the household median income level is also high.
    - Secondly, most houses with high values are grouped at southwestern part of LA county and a large part of them are located along the coastline while houses with low values are clustered at southestern part except few houses located at the north.
    - Thirdly, there is no clear pattern between walk score and housing value.Because the correlation coefficient between walk score and housing value is 0.13 which is too small to be counted.

   Except for the supposed ones in Milestone1, there are more outcomes. 
    - On one hand, there are strong correlations between household median income level and other factors: housing values, population density and violent crime rate. In areas where household median income is less than 60,000 USD , and the housing value is less than 1000,000 USD, population density is very high and violent crime rate is usually grater than 50%.For reference, the US average is 22.7%.Generally speaking, with the increase of income level and housing value, violent crime rate is decreasing but this phenomenon can not be applied to population density. 
    - On the other hand, though less significant, correlations between population density, walk score, violent crime rate and income level are also worth analyzing. In areas where population density are high, walk scores and violent crime rates are also high and vise versa.However,income level is higher in areas where population density is low.

7. **What difficulties did you have in completing the project?**

    According to Walk Score API documentation, latitudes and longitudes are required. Thus, collecting coordinates are required. Zillow API is then used.However, response from the data request is not neat and different from the format that we usually see. It takes me some time to figure out how to make a wise loop to find needed data as well as reduce repeat of code. 
    
    Also, since there are five factors involved, presenting these five variables in one graph is somehow impractical. I have tried several methods like a 3D scatter diagram but the final result is very messy. To avoid a messy situation, I decide to check the relevancy between different factors and then analyze the most relevant ones.

8. **What skills did you wish you had while you were doing the project?**
    
    Based on the current model, I think it will be better if a front-end interactive portal can be designed and implemented. To be more specific, this portal is used for users to search housing value and violent crime rate based on zip code or community name. Also, based on user’s income level, the system will return the information about his or her affordable houses. Skills like how to build a client server in Pyhon3 is needed.
    
    Besides, I also want to learn skills like how to design and construct a project in a more organized and efficient way.For instance, I do not know how to build classes and objects in a wise way.This skill can shorten my code lines and make it more efficient for reuse.Also, I want to know more skills about how to clean data in a wise way. 

9. **What would you do “next” to expand or augment the project?**
    
    To expand the project, more data are definitely needed.In the next step, I will check more websites that offer APIs with more accessible data. Also, when talking about site-choosing, more environmental factors are also of high relevance like are there any hospitals, schools, parks or food outlets around. In addition, algorithm can be applied to calculate different factors int one index to suggest the living environment condition.

In [1]:
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import folium
from folium import plugins
from folium.plugins import HeatMap
import webbrowser
import os
%matplotlib inline

## Graph1: Correlation Matrix

In [2]:
def correlation_matrix():
    conn = sqlite3.connect('final_project.db')
    cur = conn.cursor()
    sql="SELECT value,walk_score,density,crime_rate,income \
        FROM Walk_score join Value Join Crime_rate Join Urban join Income on \
        Walk_score.zipcode_id=Value.zipcode_id and Value.zipcode_id=Crime_rate.zipcode_id \
        and Value.zipcode_id=Urban.zipcode_id and Value.zipcode_id=Income.zipcode_id \
        where value!=-1"
    data = pd.read_sql(sql, conn)
    
    plt.figure(figsize=(6,5),dpi=100)
    columns=["Housing value","Walk score","Populaiton density","Crime rate","Household median income"]
    sns.heatmap(data.corr()
                ,xticklabels=columns
                ,yticklabels=columns
                ,cmap="RdYlGn"
                ,center=0
                ,annot=True
            
               )

    plt.xticks(rotation=45,horizontalalignment="right")
    plt.savefig("correlation matrix.png", bbox_inches='tight')
    plt.show()

In [13]:
correlation_matrix()

Since I want to find out the relation between income, housing value and environmental factors, scatter diagram is a perfect choice. However, there are five variables involved: housing value, median household income, walk score, population density and crime rate. We may loose the point and focus if these five factors are going to be plotted on a same graph.Thus, I decided to figure out how relevant these factors are.

Correlation matrix is used to show correlation coefficients between variables. Each cell in the graph shows the correlation between two variables.Higher the absolute value is, higher the correlation is.
From the values and colors we can tell that there are strong correlations between household median income level and another three factors: housing values, population density and crime rate. Though weaker, correlations between population density, walk score, crime rate and income level also exist.
It is confirmed with the conjestion in milestone1 that there is a positive correlation between household median income and housing value. However, the correlation coefficient between walk score and housing value is 0.13 which is too small to be counted. Thus, the conclusion is, there is no clear pattern between these two factors.
    

## Graph2-5: Scatter Diagrams

In [12]:
def data_correlation(sql,cm,name):
    conn=sqlite3.connect("final_project.db")
    cur=conn.cursor()
    cur.execute(sql)
    #cur.execute('SELECT value,income,density,crime_rate FROM Income join Value Join Crime_rate Join Urban on Income.zipcode_id=Value.zipcode_id and Value.zipcode_id=Crime_rate.zipcode_id and Value.zipcode_id=Urban.zipcode_id where value!=-1 and crime_rate!=-1 ORDER BY value ASC limit 30')
    info_list=cur.fetchall()
    communities=[]
    for info in info_list:
        value=info[0]
        cur.execute('SELECT zipcode_id FROM Value WHERE value=?', (value,))
        zipcode_id=cur.fetchone()
        cur.execute('SELECT community FROM Community WHERE zipcode_id=?', zipcode_id)
        community=cur.fetchone()[0]
        i=str(info_list.index(info)+1)
        community=i+"."+community
        communities.append(community)
    values = []
    incomes=[]
    popu_density=[]
    crime_rates=[]
    for info in info_list:
        value=info[0]
        values.append(value)
        income=info[1]
        incomes.append(income)
        density=info[2]
        popu_density.append(density/10)
        crime_rate=info[3]
        crime_rates.append(crime_rate)
    #画图
    plt.figure(num=3, figsize=(15, 15))
   
    sc=plt.scatter(incomes,values,s=popu_density,c=crime_rates,vmin=0, vmax=70,cmap=cm,label='density',alpha=0.5)#颜色代表犯罪率
    if name!="All data":
        for i in range(len(communities)):
            plt.text(incomes[i],values[i],s=communities[i],horizontalalignment="center")
    
    plt.title("Correlation between income level, hosing value and other factors")
    plt.xlabel("Household median income(USD)")
    plt.ylabel("Housing value(USD,estimated by Zillow")
            
    cb=plt.colorbar()
    cb.set_label("Crime rate")
    plt.legend() 
    plt.savefig(f"{name}.png", dpi=100,bbox_inches='tight')
    plt.show()
    
    conn.close()

In [14]:
sql='SELECT value,income,density,crime_rate \
    FROM Value Join Crime_rate Join Urban join Income on \
    Income.zipcode_id=Crime_rate.zipcode_id \
    and Income.zipcode_id=Urban.zipcode_id and Income.zipcode_id=Value.zipcode_id\
    where value!=-1'
cm="Reds"
name="All data"
data_correlation(sql,cm,name)

Graph2:Larger the point is,higher the populaiton density is.In areas where household median income is less than 60000 USD and the housing value is less than 1,000,000 USD, population density is very high and crime rate is usually grater than 50%.Generally speaking, with the increase of income level and housing value, crime rate is in decrease but this phenomenon can not be applied to populaiton density.

In [15]:
sql='SELECT value,income,density,crime_rate FROM \
    Income join Value Join Crime_rate Join Urban on \
    Income.zipcode_id=Value.zipcode_id and Income.zipcode_id=Crime_rate.zipcode_id and Income.zipcode_id=Urban.zipcode_id \
    where value!=-1 and crime_rate!=-1 ORDER BY value ASC limit 20'
cm="winter_r"
name="lowest20"
data_correlation(sql,cm,name)

Graph3:The graph shows the 20 lowest housing values in Los Angeles County.Community name is labeled on each point.There is no clear pattern between the income level and housing values. From the top left part of the corner we can tell that, though the income level is low, housing values are relatively high. However,in these 20 areas, crime rates and population density are both very high.

In [16]:
sql='SELECT value,income,density,crime_rate FROM \
    Income join Value Join Crime_rate Join Urban on \
    Income.zipcode_id=Value.zipcode_id and Income.zipcode_id=Crime_rate.zipcode_id and Income.zipcode_id=Urban.zipcode_id \
    where value!=-1 and crime_rate!=-1 ORDER BY value DESC limit 20'
cm="autumn_r"
name="highest20"
data_correlation(sql,cm,name)

Graph4:Compared with graph4, pattern of the 20 highest housing values is clearer.Generally speaking, with the increase of income level, hosuing values are also increasing.Crime rates are lower than 30% at most places. However, population density in some areas are also high.The gap betweent the most expensive house and the 20th most expensive house is about $3,000,000, which is very surprising.

In [8]:
def density_correlation():
    conn=sqlite3.connect("final_project.db")
    cur=conn.cursor()
    
    sql='SELECT walk_score,income,density,crime_rate FROM \
        Income join Walk_score Join Crime_rate Join Urban on \
        Income.zipcode_id=Urban.zipcode_id and Walk_score.zipcode_id=Urban.zipcode_id and Crime_rate.zipcode_id=Urban.zipcode_id \
        where crime_rate!=-1 '
    cur.execute(sql)
    info_list=cur.fetchall()
    
    scores = []
    incomes=[]
    popu_density=[]
    crime_rates=[]
    for info in info_list:
        score=info[0]
        scores.append(score)
        income=info[1]
        incomes.append(income/100)
        density=info[2]
        popu_density.append(density)
        crime_rate=info[3]
        crime_rates.append(crime_rate)
    #print(popu_density)
    #画图
    plt.figure(num=3, figsize=(15, 15))
    cm="summer_r"
   
    sc=plt.scatter(popu_density,scores,s=incomes,c=crime_rates,vmin=0, vmax=70,cmap=cm,label='income',alpha=0.5)#颜色代表crime rates
    
    plt.title("Correlation between pupulation density,income level,crime rates and walk score")
    plt.xlabel("Population Density")
    plt.ylabel("Walk Score")
            
    cb=plt.colorbar()
    cb.set_label("Crime rate")
    plt.legend() 
    plt.savefig(f"Density and other factors.png", dpi=80,bbox_inches='tight')
    plt.show()
    
    conn.close()

In [17]:
density_correlation()

Graph5:Shape of each point represents household median income level.In areas where population density are high, walk scores and crime rates are also high and vise versa.However,income level is higher in areas where population density is low.

## Graph6:Distribution Map

In [10]:
def house_distribution(sql1,sql2):
    
    LA_COORDINATES = (34.05, -118.24)

    conn = sqlite3.connect('final_project.db')
    cur = conn.cursor()
    
    value_data = pd.read_sql(sql1, conn)
    value=np.array(value_data["value"][0:len(value_data)])
    lat = np.array(value_data["latitude"][0:len(value_data)])
    lon = np.array(value_data["longitude"][0:len(value_data)])
   
    value_map= folium.Map(location=LA_COORDINATES,zoom_start=10)
    for i in range(50):
        folium.Marker(location=[lat[i],lon[i]],popup=f"Value:${value[i]}",icon=folium.Icon(color="red", icon='home')).add_to(value_map)
      
    value_data = pd.read_sql(sql2, conn)
    value=np.array(value_data["value"][0:len(value_data)])
    lat = np.array(value_data["latitude"][0:len(value_data)])
    lon = np.array(value_data["longitude"][0:len(value_data)])
    for i in range(50):
        folium.Marker(location=[lat[i],lon[i]],popup=f"Value:${value[i]}",icon=folium.Icon(color="blue", icon='home')).add_to(value_map)
  
    path=os.getcwd()
    file_path = f"{path}\spatial distribution.html"
    value_map.save(file_path)
    webbrowser.open(file_path)
    display(value_map)
    conn.close()

In [20]:
sql1="select value,latitude,longitude,zip_code from \
    Value join Zipcode on Value.zipcode_id==Zipcode.id\
    where value!=-1 order by value DESC limit 50"
sql2="select value,latitude,longitude,zip_code from \
    Value join Zipcode on Value.zipcode_id==Zipcode.id \
    where value!=-1 order by value ASC limit 50"
house_distribution(sql1,sql2)

This map is interactive, you can zoom in or zoom out the map as you wish. You can click each icon to check the housing value at each marked location.

On this map, red icons and blue icons mark 50 house locations with highest and lowest housing values in Los Angeles County respectively.
Spatial distribution of these houeses are interesting.Most houses with high values are grouped at southwestern part of LA county and a large part of them are along the coastline while houses with low values are clustered at southestern part except few houses located at the north.